In [1]:
import datetime
import pandas as pd
import random
import numpy as np
import seaborn as sns

In [2]:
rng = np.random.default_rng(123)

start_year = 1950
end_year = 2020
portfolios = ["First", "Second", "Third", "Fourth", "Fifth"]
portfolios_list = []
dates_list = []


for portfolio in portfolios:
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            date = datetime.date(year, month, 1)
            dates_list.append(date)
            portfolios_list.append(portfolio)

random_returns = rng.uniform(low=(-0.1 / 12), high=(0.3 / 12), size=len(dates_list))


fake_returns = pd.DataFrame(
    {"Portfolio": portfolios_list, "date": dates_list, "ret": random_returns}
)


In [3]:
fake_returns

,Portfolio,date,ret
0,First,1950-01-01,0.014412
1,First,1950-02-01,-0.006539
2,First,1950-03-01,-0.000988
3,First,1950-04-01,-0.002188
4,First,1950-05-01,-0.002470
...,...,...,...
4255,Fifth,2020-08-01,0.002473
4256,Fifth,2020-09-01,0.004191
4257,Fifth,2020-10-01,-0.007476
4258,Fifth,2020-11-01,0.005397


In [4]:
# Some pseudo code

# Input variables (these require input from the user, MANDATORY)

submitted_income = 85000
income_growth = 0.035

monthly_income = submitted_income/12
monthly_growth = 1+ (income_growth/12)

start_savings = 25
retirement_start = 65
death_year = 85

month_start_savings = start_savings*12
month_retirement_start = retirement_start*12
death_month = death_year*12



# I think I just want these to be static
save_rate = 0.10
consumption_rate = 0.04

In [5]:
total_monthly_incomes = []

for t in fake_returns.index[month_start_savings:month_retirement_start]:
    current_month = t-month_start_savings
    income_for_month = monthly_income*(monthly_growth**current_month)
    total_monthly_incomes.append(income_for_month)


In [33]:
# Apply mapping to DataFrame's index to create a month column that resets for each portfolio
fake_returns['month'] = fake_returns.groupby('Portfolio').cumcount()

# Map the total_monthly_incomes to each row based on the month
income_series = pd.Series(total_monthly_incomes, index=range(month_start_savings, month_retirement_start))
fake_returns['income'] = fake_returns['month'].map(income_series)

# Optional: Remove the month column if it's no longer needed
#fake_returns.drop(columns='month', inplace=True)


In [30]:
savings = []

for t in fake_returns.index[month_start_savings:month_retirement_start]:
    current_month = t - month_start_savings
    if current_month > 0:
        monthly_savings = (
            save_rate * total_monthly_incomes[current_month] + savings[current_month - 1]
        )
        savings.append(monthly_savings)
    else:
        monthly_savings = save_rate * total_monthly_incomes[current_month]
        savings.append(monthly_savings)

In [34]:
fake_returns['savings'] = fake_returns.groupby('Portfolio')['ret']

,Portfolio,date,ret,month,income
0,First,1950-01-01,0.014412,0,NaN
1,First,1950-02-01,-0.006539,1,NaN
2,First,1950-03-01,-0.000988,2,NaN
3,First,1950-04-01,-0.002188,3,NaN
4,First,1950-05-01,-0.002470,4,NaN
...,...,...,...,...,...
4255,Fifth,2020-08-01,0.002473,847,NaN
4256,Fifth,2020-09-01,0.004191,848,NaN
4257,Fifth,2020-10-01,-0.007476,849,NaN
4258,Fifth,2020-11-01,0.005397,850,NaN


In [ ]:
fake_returns.query("Portfolio=='Fifth'")

,Portfolio,date,ret,income
3408,Fifth,1950-01-01,0.007492,7083.333333
3409,Fifth,1950-02-01,0.000653,7103.993056
3410,Fifth,1950-03-01,-0.006861,7124.713035
3411,Fifth,1950-04-01,0.016663,7145.493448
3412,Fifth,1950-05-01,-0.003020,7166.334471
...,...,...,...,...
4255,Fifth,2020-08-01,0.002473,NaN
4256,Fifth,2020-09-01,0.004191,NaN
4257,Fifth,2020-10-01,-0.007476,NaN
4258,Fifth,2020-11-01,0.005397,NaN


In [35]:
def calculate_savings(group):
    # Ensure the group is sorted by date
    group = group.sort_values('date')
    # Initialize savings list
    savings = []
    # Initialize a variable to keep track of last savings
    last_savings = 0
    for _, row in group.iterrows():
        # Calculate current savings
        current_savings = save_rate * row['income'] + last_savings * (1 + row['ret'])
        # Append to list
        savings.append(current_savings)
        # Update last savings to current
        last_savings = current_savings
    # Assign the savings list to the 'savings' column
    group['savings'] = savings
    return group

In [36]:
test_df = fake_returns.groupby('Portfolio').apply(calculate_savings).reset_index(drop=True)


In [43]:
test_df.iloc[month_start_savings:month_retirement_start]

,Portfolio,date,ret,month,income,savings
300,Fifth,1975-01-01,-0.005944,300,7083.333333,NaN
301,Fifth,1975-02-01,-0.006775,301,7103.993056,NaN
302,Fifth,1975-03-01,0.014836,302,7124.713035,NaN
303,Fifth,1975-04-01,0.010837,303,7145.493448,NaN
304,Fifth,1975-05-01,0.012035,304,7166.334471,NaN
...,...,...,...,...,...,...
775,Fifth,2014-08-01,0.022896,775,28251.450292,NaN
776,Fifth,2014-09-01,0.000022,776,28333.850355,NaN
777,Fifth,2014-10-01,-0.008245,777,28416.490752,NaN
778,Fifth,2014-11-01,0.017016,778,28499.372184,NaN
